# Carga de librerías

In [1]:
# Básicas
import pandas as pd
import numpy as np
import time
import os
from tqdm import tqdm
import re
# Scraping
from selenium import webdriver
from selenium.webdriver.common.by import By